In [1]:
#imports
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import sys
import pandas as pd
import time
import math
import random

In [2]:
def calculate_minimum_trucks_from_demand(demand_from_nodes, avg_vehicle_capacity):
    total = 0
    with open(demand_from_nodes, 'r') as f:
        for line in f:
            total = total + int(line)
        f.close()
    
    minimum_num = math.floor(total / avg_vehicle_capacity)
    return minimum_num

def vehicle_demand_initialiser(total_number_of_nodes, demand_from_nodes):
    with open(demand_from_nodes, 'w') as f:
        f.write("0" + '\n')
        for i in range(total_number_of_nodes - 1):
            demand = random.randint(1, 10)
            f.write(str(demand) + '\n')
        f.close()
        
def vehicle_time_window_initialiser(total_number_of_nodes, nodes_time_window):
    
    with open(nodes_time_window, 'w') as f:
        f.write("1"+ ' ' + "300" + '\n')
        for i in range(total_number_of_nodes - 1):
            begin = random.randint(1,10)
            end   = random.randint(begin+1, 100)
            f.write(str(begin) + ' ' + str(end) + '\n')
        f.close()

In [3]:
def create_data_model(demand_from_nodes, nodes_time_window):
    """Stores the data for the problem."""

    ### INITIALISE DATA STRUCTURE
    data = {}
    data['demands'] = []
    data['time_matrix']  = []
    data['time_windows'] = []
    data['num_vehicles'] = 2
    data['depot'] = 0
    data['vehicle_capacities'] = [50] * data['num_vehicles']


    ### READ DEMAND FOR EACH NODE FROM FILE
    with open(demand_from_nodes, 'r') as f:
        for line in f:
            data['demands'].append(int(line.rstrip()))
        f.close()
    
    number_of_nodes = len(data['demands'])
    ### READ TRAVEL TIME DATA FROM FILE
    time_matrix_data = pd.read_csv('site_time.csv')
    time_matrix_data = time_matrix_data.iloc[:number_of_nodes, 1:number_of_nodes+1]
    time_matrix      = time_matrix_data.values.tolist()
    for site_time_data_row in time_matrix:
        data['time_matrix'].append(site_time_data_row)
    
    ### READ TIME WINDOW DATA FROM FILE
    with open(nodes_time_window, 'r') as f:
        for line in f:
            window_start, window_end = map(int,line.rstrip().split())
            data['time_windows'].append((window_start, window_end))
        f.close()
        
    return data


In [4]:
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))


In [5]:
def wrapper(number_of_vehicles, vehicle_capacity, demand_from_nodes, nodes_time_window):
    
    # Instantiate the data problem.
    data = create_data_model(demand_from_nodes, nodes_time_window)
    data['num_vehicles'] = number_of_vehicles
    data['vehicle_capacities'] = [vehicle_capacity] * number_of_vehicles


    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

        
    # Time Callback and constraints
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    
    # Using time transit callback as optimisation parameter
    routing.SetArcCostEvaluatorOfAllVehicles(time_callback_index)

    
    ### Smaller values limit the travel of vehicles, the following values have no affect on time windows
    ### Large values makes sure that we get the best solution for now
    '''Time Window Constraint'''
    time = 'Time'
    routing.AddDimension(
        time_callback_index,
        10000,  # allow waiting time
        10000,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        # Require that a vehicle must visit a location during the location's time window.
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
    
    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))
    
    
    
    # Demand callback and constaints
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')


    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        print("Vehicles Required: {}".format(number_of_vehicles))
        print("-"*40)
        return 1
    else:
        print("Current Number of Vehicles = {}".format(number_of_vehicles))
        print("No Solution Yet")
        print("-"*40)
        print("Incrementing number of vehicles")
        return 0

In [6]:
def main():
    
    """ Files Required
            - site_time.csv (time to travel between any two nodes)
            - Node_Demands.txt (can be generated)
            - nodes_time_window.txt (can be generated)
            
            To change number of nodes, just edit the variable below and regenerate demand and time windows.
            Solution isn't guaranteed when new time windows are created
            
            Preferably change the name of text file so that previous data isn't lost.
    """
    
    
    demand_from_nodes  = "Node_Demands.txt"
    nodes_time_window  = "nodes_time_window.txt" # Filename containing time window for each node
    number_of_nodes    = 20
    
#     vehicle_demand_initialiser(number_of_nodes, demand_from_nodes)
#     vehicle_time_window_initialiser(number_of_nodes, nodes_time_window)
    
    vehicle_capacity   = 30
    current_num_of_trucks = calculate_minimum_trucks_from_demand(demand_from_nodes, vehicle_capacity)
    print("minimum_num_of_vehicles = {}".format(current_num_of_trucks) + '\n' + '-'*40)
    solution = wrapper(current_num_of_trucks, vehicle_capacity, demand_from_nodes, nodes_time_window)
    while not solution:
        current_num_of_trucks += 1
        solution = wrapper(current_num_of_trucks, vehicle_capacity, demand_from_nodes, nodes_time_window)
        if(current_num_of_trucks > 100):
            break
    
    #TODO: Add binary search instead of linear search.

In [7]:
if __name__ == '__main__':
    main()

minimum_num_of_vehicles = 4
----------------------------------------
Current Number of Vehicles = 4
No Solution Yet
----------------------------------------
Incrementing number of vehicles
Route for vehicle 0:
 0 ->  8 ->  3 ->  4 ->  5 -> 0
Distance of the route: 14m

Route for vehicle 1:
 0 ->  10 ->  19 ->  16 ->  14 ->  9 -> 0
Distance of the route: 25m

Route for vehicle 2:
 0 ->  6 ->  12 ->  1 ->  7 -> 0
Distance of the route: 19m

Route for vehicle 3:
 0 ->  18 ->  15 -> 0
Distance of the route: 7m

Route for vehicle 4:
 0 ->  17 ->  13 ->  11 ->  2 -> 0
Distance of the route: 13m

Total Distance of all routes: 78m
Vehicles Required: 5
----------------------------------------
